In [1]:
from tensoraerospace.agent.pid import PIDController
import numpy as np
from ray import train, tune

from tensoraerospace.envs.f16.linear_longitudial import LinearLongitudinalF16
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

import gymnasium as gym 
from tensoraerospace.benchmark.function import overshoot, settling_time, static_error

In [2]:
dt = 0.01  # Дискретизация
tp = generate_time_period(tn=20, dt=dt) # Временной периуд
tps = convert_tp_to_sec_tp(tp, dt=dt)
number_time_steps = len(tp) # Количество временных шагов
reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1]) # Заданный сигнал

In [5]:
def env_optimization(ki, kp, kd):
    """
    Оптимизация среды моделирования для настройки коэффициентов ПИД-регулятора.

    Эта функция оптимизирует параметры ПИД-регулятора (ки, кп, кд), используя модельную среду.
    Она оценивает качество настройки по критериям статической ошибки, перерегулирования и времени установления.

    Args:
        ki (float): Коэффициент интегральной составляющей.
        kp (float): Коэффициент пропорциональной составляющей.
        kd (float): Коэффициент дифференциальной составляющей.

    Returns:
        float: Суммарная оценка качества настройки регулятора.

    """

    # Инициализация истории и настройка параметров времени
    hist = []
    dt = 0.01
    tp = generate_time_period(tn=25, dt=dt)
    tps = convert_tp_to_sec_tp(tp, dt=dt)
    number_time_steps = len(tp)

    # Создание заданного сигнала
    reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1])

    # Настройка модельной среды
    env = gym.make('LinearLongitudinalB747-v0',
                number_time_steps=number_time_steps, 
                initial_state=[[0],[0],[0],[0]],
                reference_signal=reference_signals,
                state_space = [ "u", "w", "q", "theta"],
                output_space = ["u", "w", "q", "theta"],
                tracking_states=["theta"], use_reward = False)
    env.reset()

    # Инициализация ПИД-регулятора
    pid = PIDController(kp=kp, ki=ki, kd=kd, dt=dt)
    xt = np.array([[np.deg2rad(0)], [0]])

    # Цикл моделирования среды
    for step in range(number_time_steps - 2):
        setpoint = reference_signals[0, step]
        hist.append(xt[0, 0])
        ut = pid.update(setpoint, xt[0, 0])
        xt, reward, terminated, truncated, info = env.step(np.array([ut.item()]))

    # Условие для досрочного прекращения оптимизации
    if 6 < max(hist):
        return 10000

    # Получение исходных сигналов для анализа
    system_signal_orig = env.unwrapped.model.get_state('alpha', to_deg=True)[:2500]
    control_signal_orig = np.rad2deg(reference_signals[0])[:2500]

    # Расчет оценочной функции
    return np.abs(static_error(control_signal_orig, system_signal_orig)) * 0.4 + \
           np.abs(overshoot(control_signal_orig, system_signal_orig)) ** 0.1 + \
           (settling_time(control_signal_orig, system_signal_orig) * dt) ** 0.9


In [6]:
env_optimization(1, 1, 1)

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:246: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'numpy.ndarray'>
  logger.warn(


28.532090510385167

In [7]:
def easy_objective(config):
    """
    Функция цели для оптимизатора.

    Принимает конфигурацию с параметрами ПИД-регулятора и возвращает оценку качества настройки.

    Args:
        config (dict): Словарь с параметрами ПИД-регулятора (ki, kp, kd).

    """

    # Извлечение гиперпараметров из конфигурации
    ki, kp, kd = config["ki"], config["kp"], config["kd"]

    # Вычисление оценки
    intermediate_score = env_optimization(ki, kp, kd)

    # Отчет о текущем значении потерь
    train.report({"mean_loss": intermediate_score})

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-20 00:23:15,715	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-20 00:23:15,931	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [8]:
# Настройка и запуск оптимизатора
tuner = tune.Tuner(
        easy_objective,
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            num_samples=40000,
        ),
        param_space={
            "ki": tune.uniform(-10, 10),
            "kp": tune.uniform(-10, 10),
            "kd": tune.uniform(-10, 10),
        },
    )

In [ ]:
# Выполнение оптимизации
results = tuner.fit()

In [10]:
# Получение лучших результатов оптимизации
results.get_best_result().config

{'ki': -9.41904814284741, 'kp': -2.419889382764948, 'kd': -1.0979980771661264}

In [11]:
# Сортировка и получение значений средних потерь
results.get_dataframe()['mean_loss'].sort_values()

26146     11.603939
30309     11.608407
30618     11.637506
2638      11.641204
18715     11.645229
            ...    
24800    163.910968
20547    164.256281
13299    164.271982
26730    164.482116
28376    164.892957
Name: mean_loss, Length: 39997, dtype: float64